In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np

In [2]:
# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

In [3]:
# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

In [4]:
#examine the data
# school_data.head()

In [5]:
#examine the data
# student_data.head()

In [6]:
# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
# school_data_complete

In [7]:
# assign summary data to variables
schools_total = school_data_complete["School ID"].nunique()
student_total = school_data_complete["Student ID"].nunique()
total_budget = school_data_complete["budget"].unique().sum()
average_math = school_data_complete["math_score"].mean(axis=0)
average_reading = school_data_complete["reading_score"].mean(axis=0)
passing_math_count = school_data_complete.math_score[(school_data_complete.math_score  >= 70)].count()
passing_math_percent = passing_math_count / student_total * 100
passing_reading_count = school_data_complete.reading_score[(school_data_complete.reading_score >= 70)].count()
passing_reading_percent = passing_reading_count / student_total * 100 
passing_both_count = len(school_data_complete[(school_data_complete['math_score'] >= 70) & (school_data_complete['reading_score'] >= 70) ])
passing_both_percent = (passing_both_count / student_total) * 100


In [8]:
# create summary dataframe
combined_df = pd.DataFrame({"Total Schools": [schools_total],
                            "Total Students": [student_total],
                            "Total Budget":   [total_budget], # is this value correct?
                            "Average Math Score": [f'{round(average_math,1)}%'],
                            "Average Reading Score": [f'{round(average_reading, 1)}%'],
                            "Students Passing Math": [f'{round(passing_math_percent, 1)}%'],
                            "Students Passing Reading": [f'{round(passing_reading_percent, 1)}%'],
                            "Overall Passing": [f'{round(passing_both_percent, 1)}%']
                           
                           })
# formatting
combined_df['Total Budget'] = combined_df['Total Budget'].astype(int).apply(lambda x: "${:,}".format(x))
combined_df['Total Students'] = combined_df['Total Students'].astype(int).apply(lambda x: "{:,}".format(x))

# show df
combined_df


,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,Students Passing Math,Students Passing Reading,Overall Passing
0,15,"39,170","$24,649,428",79.0%,81.9%,75.0%,85.8%,65.2%


In [9]:
# Group by schools
groupby_school = school_data_complete.groupby(['school_name'])



# Create series for df
school_type = groupby_school['type'].unique()
size = groupby_school['size'].unique()
budget_per_school = groupby_school['budget'].mean()
average_math_per_school = groupby_school['math_score'].mean() 
average_reading_per_school = groupby_school['reading_score'].mean()
passing_math = school_data_complete.loc[school_data_complete['math_score'] >= 70, ['school_name','math_score']]
passing_math_counts = passing_math['school_name'].value_counts()
passing_reading = school_data_complete.loc[school_data_complete['reading_score'] >= 70, ['school_name','reading_score']]
passing_reading_counts = passing_reading['school_name'].value_counts()
passing_both = school_data_complete.loc[(school_data_complete['math_score'] >= 70) & (school_data_complete['reading_score'] >= 70), ['school_name','math_score']]
passing_both_counts = passing_both['school_name'].value_counts()
passing_math_counts = passing_math_counts.sort_index()
passing_reading_counts = passing_reading_counts.sort_index()
passing_both_counts = passing_both_counts.sort_index()


In [10]:
# concatenate series into df
schools_summary_df = pd.concat([school_type, size, budget_per_school, average_math_per_school, average_reading_per_school], axis = 1)

In [11]:
# create other columns,format
schools_summary_df['Passing Math Percent'] = round(((passing_math_counts / size) * 100).astype(float), 1)
schools_summary_df['Passing Reading Percent'] = round(((passing_reading_counts / size) * 100).astype(float), 1)
schools_summary_df['Passing Both Percent'] = round(((passing_both_counts / size) * 100).astype(float), 1)
schools_summary_df['type'] = schools_summary_df['type'].str[0]
schools_summary_df['size'] = schools_summary_df['size'].astype(int)
schools_summary_df['Budget per student'] = schools_summary_df['budget'] / schools_summary_df['size']

# formatting
schools_summary_df = schools_summary_df.rename(columns={'type':'School Type', 
                                                        'size': 'Size', 
                                                        'budget': 'Total Budget',
                                                        'math_score': 'Average Math Score (%)',
                                                        'reading_score': 'Average Reading Score (%)',
                                                        'Passing Reading Percent': 'Passing Reading (%)',
                                                        'Passing Math Percent': 'Passing Math (%)',
                                                        'Passing Both Percent': 'Passing Both (%)',
                                                        'Budget per student': 'Budget per Student ($)'})

schools_summary_df['Total Budget'] = schools_summary_df['Total Budget'].astype(int).apply(lambda x: "${:,}".format(x))
schools_summary_df['Average Math Score (%)'] = schools_summary_df['Average Math Score (%)'].round(1)
schools_summary_df['Average Reading Score (%)'] = schools_summary_df['Average Reading Score (%)'].round(1)

In [12]:
# print df
schools_summary_df()

,School Type,Size,Total Budget,Average Math Score (%),Average Reading Score (%),Passing Math (%),Passing Reading (%),Passing Both (%),Budget per Student ($)
school_name,,,,,,,,,
Bailey High School,District,4976,"$3,124,928",77.0,81.0,66.7,81.9,54.6,628.0
Cabrera High School,Charter,1858,"$1,081,356",83.1,84.0,94.1,97.0,91.3,582.0
Figueroa High School,District,2949,"$1,884,411",76.7,81.2,66.0,80.7,53.2,639.0
Ford High School,District,2739,"$1,763,916",77.1,80.7,68.3,79.3,54.3,644.0
Griffin High School,Charter,1468,"$917,500",83.4,83.8,93.4,97.1,90.6,625.0


In [13]:
# new dfs sorted by passing %
top_five = schools_summary_df.sort_values('Passing Both (%)', ascending = False)
top_five.head()

,School Type,Size,Total Budget,Average Math Score (%),Average Reading Score (%),Passing Math (%),Passing Reading (%),Passing Both (%),Budget per Student ($)
school_name,,,,,,,,,
Cabrera High School,Charter,1858,"$1,081,356",83.1,84.0,94.1,97.0,91.3,582.0
Thomas High School,Charter,1635,"$1,043,130",83.4,83.8,93.3,97.3,90.9,638.0
Griffin High School,Charter,1468,"$917,500",83.4,83.8,93.4,97.1,90.6,625.0
Wilson High School,Charter,2283,"$1,319,574",83.3,84.0,93.9,96.5,90.6,578.0
Pena High School,Charter,962,"$585,858",83.8,84.0,94.6,95.9,90.5,609.0


In [14]:
# new df sorted be lowest passing #
bottom_five = schools_summary_df.sort_values('Passing Both (%)', ascending = True)
bottom_five.head()

,School Type,Size,Total Budget,Average Math Score (%),Average Reading Score (%),Passing Math (%),Passing Reading (%),Passing Both (%),Budget per Student ($)
school_name,,,,,,,,,
Rodriguez High School,District,3999,"$2,547,363",76.8,80.7,66.4,80.2,53.0,637.0
Figueroa High School,District,2949,"$1,884,411",76.7,81.2,66.0,80.7,53.2,639.0
Hernandez High School,District,4635,"$3,022,020",77.3,80.9,66.8,80.9,53.5,652.0
Huang High School,District,2917,"$1,910,635",76.6,81.2,65.7,81.3,53.5,655.0
Johnson High School,District,4761,"$3,094,650",77.1,81.0,66.1,81.2,53.5,650.0


In [15]:
# series sorted by grade
ninth_grade_math = school_data_complete.loc[school_data_complete['grade'] == "9th", ['school_name', 'math_score']]
tenth_grade_math = school_data_complete.loc[school_data_complete['grade'] == "10th", ['school_name', 'math_score']]
eleventh_grade_math = school_data_complete.loc[school_data_complete['grade'] == "11th", ['school_name', 'math_score']]
twelfth_grade_math = school_data_complete.loc[school_data_complete['grade'] == "12th", ['school_name', 'math_score']]

In [16]:
# group series by school name (math)
ninth_grade_math = round(ninth_grade_math.groupby('school_name').mean(),1)
tenth_grade_math = round(tenth_grade_math.groupby('school_name').mean(),1)
eleventh_grade_math = round(eleventh_grade_math.groupby('school_name').mean(),1)
twelfth_grade_math = round(twelfth_grade_math.groupby('school_name').mean(),1)


In [17]:
# combine series in to df (math)

math_score_by_grade_df = pd.DataFrame({'9th': ninth_grade_math['math_score'],
                                      '10th': tenth_grade_math['math_score'],
                                       '11th': eleventh_grade_math['math_score'],
                                       '12th': twelfth_grade_math['math_score']
                                      })

math_score_by_grade_df

,9th,10th,11th,12th
school_name,,,,
Bailey High School,77.1,77.0,77.5,76.5
Cabrera High School,83.1,83.2,82.8,83.3
Figueroa High School,76.4,76.5,76.9,77.2
Ford High School,77.4,77.7,76.9,76.2
Griffin High School,82.0,84.2,83.8,83.4
Hernandez High School,77.4,77.3,77.1,77.2
Holden High School,83.8,83.4,85.0,82.9
Huang High School,77.0,75.9,76.4,77.2
Johnson High School,77.2,76.7,77.5,76.9


In [18]:
# series sorted by grade (reading)
ninth_grade_reading = school_data_complete.loc[school_data_complete['grade'] == "9th", ['school_name', 'reading_score']]
tenth_grade_reading = school_data_complete.loc[school_data_complete['grade'] == "10th", ['school_name', 'reading_score']]
eleventh_grade_reading = school_data_complete.loc[school_data_complete['grade'] == "11th", ['school_name', 'reading_score']]
twelfth_grade_reading = school_data_complete.loc[school_data_complete['grade'] == "12th", ['school_name', 'reading_score']]

In [19]:
# group series by school name (reading)
ninth_grade_reading = round(ninth_grade_reading.groupby('school_name').mean(),1)
tenth_grade_reading = round(tenth_grade_reading.groupby('school_name').mean(),1)
eleventh_grade_reading = round(eleventh_grade_reading.groupby('school_name').mean(),1)
twelfth_grade_reading = round(twelfth_grade_reading.groupby('school_name').mean(),1)

In [20]:
# combine series in to df (reading)

reading_score_by_grade_df = pd.DataFrame({'9th': ninth_grade_reading['reading_score'],
                                      '10th': tenth_grade_reading['reading_score'],
                                       '11th': eleventh_grade_reading['reading_score'],
                                       '12th': twelfth_grade_reading['reading_score']
                                      })

reading_score_by_grade_df

,9th,10th,11th,12th
school_name,,,,
Bailey High School,81.3,80.9,80.9,80.9
Cabrera High School,83.7,84.3,83.8,84.3
Figueroa High School,81.2,81.4,80.6,81.4
Ford High School,80.6,81.3,80.4,80.7
Griffin High School,83.4,83.7,84.3,84.0
Hernandez High School,80.9,80.7,81.4,80.9
Holden High School,83.7,83.3,83.8,84.7
Huang High School,81.3,81.5,81.4,80.3
Johnson High School,81.3,80.8,80.6,81.2


In [21]:
# create bins
bins = [0, 585, 630, 645, 680]
my_labels = ['<$585', '$585-630', '$630-645', '$645-680']

# cut together
schools_summary_df['Spend Range (per student)'] = pd.cut(schools_summary_df['Budget per Student ($)'], bins, labels=my_labels)



In [22]:
# groupby budget group
budget_bins_df = schools_summary_df.groupby('Spend Range (per student)').mean()
budget_bins_df = budget_bins_df[['Average Math Score (%)','Average Reading Score (%)','Passing Math (%)','Passing Reading (%)', 'Passing Both (%)']]
budget_bins_df

,Average Math Score (%),Average Reading Score (%),Passing Math (%),Passing Reading (%),Passing Both (%)
Spend Range (per student),,,,,
<$585,83.475,83.950000,93.45,96.600000,90.35
$585-630,81.900,83.125000,87.15,92.700000,81.40
$630-645,78.500,81.600000,73.50,84.375000,62.85
$645-680,77.000,81.033333,66.20,81.133333,53.50


In [23]:
# create bins for school size
bins2 = [0, 1000, 2000, 5000]
my_labels2 = ['Small (<1000)', 'Medium (1000-2000)', 'Large (2000-5000)']

# cut together
size_bins_df = schools_summary_df['School Size'] = pd.cut(schools_summary_df['Size'], bins2, labels=my_labels2)

size_bins_df = schools_summary_df.groupby('School Size').mean()
size_bins_df = size_bins_df[['Average Math Score (%)','Average Reading Score (%)','Passing Math (%)','Passing Reading (%)', 'Passing Both (%)']]
size_bins_df

,Average Math Score (%),Average Reading Score (%),Passing Math (%),Passing Reading (%),Passing Both (%)
School Size,,,,,
Small (<1000),83.8000,83.9000,93.5500,96.10,89.850
Medium (1000-2000),83.4000,83.8600,93.6000,96.78,90.600
Large (2000-5000),77.7375,81.3375,69.9875,82.75,58.275


In [24]:
school_type_df = schools_summary_df.groupby('School Type').mean()
school_type_df = school_type_df[['Average Math Score (%)','Average Reading Score (%)','Passing Math (%)','Passing Reading (%)', 'Passing Both (%)']]
school_type_df

,Average Math Score (%),Average Reading Score (%),Passing Math (%),Passing Reading (%),Passing Both (%)
School Type,,,,,
Charter,83.487500,83.887500,93.625000,96.575000,90.412500
District,76.942857,80.957143,66.571429,80.785714,53.657143
